In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install selenium
!pip install -q google-colab-selenium
!pip install nltk


#untuk kebutuhan pemrosesan teks bahasa indonesia pada tutorial ini
!pip install nlp-id
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib3-2.4.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 37.1 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=742a768c27c2e4cc22167758875dbb1b22bba1dfab41421c3a9b31810c1913ef
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
  Attempting uninstall: huggingface-hub
    Found exist

In [2]:
# Import library
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import string
import nltk

# Download stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# Link website
url = "https://www.scrapethissite.com/pages/simple/"
response = requests.get(url)

# Parse HTML
soup = BeautifulSoup(response.content, 'html.parser')

# Ambil semua div country
countries = soup.find_all("div", class_="country")

# Simpan data
data = []

for country in countries:
    name = country.find("h3", class_="country-name").text.strip()
    capital = country.find("span", class_="country-capital").text.strip()
    population = country.find("span", class_="country-population").text.strip()
    area = country.find("span", class_="country-area").text.strip()

    data.append({
        "name": name,
        "capital": capital,
        "population": population,
        "area": area
    })

# Buat DataFrame
df = pd.DataFrame(data)

# Tampilkan beberapa data teratas
df.head()

,name,capital,population,area
0,Andorra,Andorra la Vella,84000,468.0
1,United Arab Emirates,Abu Dhabi,4975593,82880.0
2,Afghanistan,Kabul,29121286,647500.0
3,Antigua and Barbuda,St. John's,86754,443.0
4,Anguilla,The Valley,13254,102.0


In [4]:
# Inisialisasi stopword & stemmer
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

# Function preprocessing
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Hapus angka
    text = re.sub(r'\d+', '', text)
    # Hapus tanda baca
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Hapus whitespace ekstra
    text = text.strip()
    # Tokenisasi
    words = text.split()
    # Hapus stopword dan stemming
    words = [stemmer.stem(w) for w in words if w not in stop_words]
    # Gabungkan kembali
    return " ".join(words)

# Terapkan ke kolom
df['name_clean'] = df['name'].apply(preprocess_text)
df['capital_clean'] = df['capital'].apply(preprocess_text)

# Tampilkan DataFrame setelah preprocessing
df.head()

,name,capital,population,area,name_clean,capital_clean
0,Andorra,Andorra la Vella,84000,468.0,andorra,andorra la vella
1,United Arab Emirates,Abu Dhabi,4975593,82880.0,unit arab emir,abu dhabi
2,Afghanistan,Kabul,29121286,647500.0,afghanistan,kabul
3,Antigua and Barbuda,St. John's,86754,443.0,antigua barbuda,st john
4,Anguilla,The Valley,13254,102.0,anguilla,valley


In [5]:
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    words = text.split()
    words = [stemmer.stem(w) for w in words if w not in stop_words]
    return " ".join(words)

# Kolom before-after lowercase
df['name_lower'] = df['name'].apply(lambda x: x.lower())
df['capital_lower'] = df['capital'].apply(lambda x: x.lower())

# Kolom final cleaned
df['name_clean'] = df['name'].apply(preprocess_text)
df['capital_clean'] = df['capital'].apply(preprocess_text)

df[['name', 'name_lower', 'name_clean', 'capital', 'capital_lower', 'capital_clean']].head()

,name,name_lower,name_clean,capital,capital_lower,capital_clean
0,Andorra,andorra,andorra,Andorra la Vella,andorra la vella,andorra la vella
1,United Arab Emirates,united arab emirates,unit arab emir,Abu Dhabi,abu dhabi,abu dhabi
2,Afghanistan,afghanistan,afghanistan,Kabul,kabul,kabul
3,Antigua and Barbuda,antigua and barbuda,antigua barbuda,St. John's,st. john's,st john
4,Anguilla,anguilla,anguilla,The Valley,the valley,valley


In [6]:
df.to_csv("HandsOnTask1.csv", index=False, encoding="utf-8-sig")
print("✅ Data berhasil disimpan ke scraped_countries_with_lowercase.csv")

✅ Data berhasil disimpan ke scraped_countries_with_lowercase.csv
